In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import pickle
import numpy as np
transformer_model=SentenceTransformer("hiiamsid/sentence_similarity_spanish_es")

In [2]:
descripciones=pd.read_csv("../database/inventario_reportes.csv", dtype=str)
print(descripciones.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1598 entries, 0 to 1597
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   ID_INVENTARIO_REPORTE  1598 non-null   object
 1   CLAVE_REP              1598 non-null   object
 2   CLAVE_PAIS             1598 non-null   object
 3   CLAVE_ENTIDADREGULADA  1598 non-null   object
 4   CLAVE_REG              1463 non-null   object
 5   CLAVE_SERIE            1461 non-null   object
 6   SUBSERIE               620 non-null    object
 7   CLAVE_GRUPO            1463 non-null   object
 8   REPORTE                1463 non-null   object
 9   CLAVE_SECCION_REP      1462 non-null   object
 10  CLAVE_VERSION_REPORTE  1463 non-null   object
 11  CLAVE_PERIODO          1390 non-null   object
 12  DESCRIPCION_ESP        1598 non-null   object
 13  CLAVE_FECHA_ENT_REP    166 non-null    object
 14  CARACTERISTICAS        194 non-null    object
 15  CLAVE_REGULACION_REP 

In [3]:
print(descripciones["DESCRIPCION_ESP"].values)

['AC INFORMACIÓN DE ACCIONISTAS' 'AD ACTIVOS EXTRAORDINARIOS COMPLEMENTO'
 'AE ACTIVOS EXTRAORDINARIOS' ...
 'Nu´mero de socios, empleados y sucursales'
 'Requerimientos de capital por riesgos'
 'Requerimientos de capital por riesgos']


In [4]:
data={}
for idx, row in descripciones.iterrows():
    embedding=transformer_model.encode(row["DESCRIPCION_ESP"])
    data[row["CLAVE_REP"]]={
        "CLAVE_REP": row["CLAVE_REP"],
        "DESCRIPCION_ESP": row["DESCRIPCION_ESP"],
        "EMBEDDING": embedding.tolist()
    }

In [5]:
# Guardamos en un pickle
with open("embeddings.pkl", "wb") as f:
    pickle.dump(data, f)
# Cargamos el pickle
with open("embeddings.pkl", "rb") as f:
    data = pickle.load(f)

In [6]:
# Dada un nuevo texto generamos su embedding y buscamos el más similar
text="En qué reportes se envía información de partes relacionadas?"
embedding=transformer_model.encode(text.replace("reportes", ""))
# Buscamos el más cercano
distances=[]
for clave, info in data.items():
    dist=np.linalg.norm(np.array(info["EMBEDDING"])-np.array(embedding))
    distances.append((dist, info))
# Ordenamos por distancia
distances.sort(key=lambda x: x[0])
print("Más cercanos:")
# Guardamos los 10 mas cercasnos (solo CLAVE_REP)
closest_ids=[]
closest_ids=[info["CLAVE_REP"] for _, info in distances[:10]]
for dist, info in distances[:10]:
    print(f"{dist:.4f} {info['CLAVE_REP']} {info['DESCRIPCION_ESP']}")
    # Guardamos los IDs de los reportes más cercanos


Más cercanos:
17.0360 MX_AF_AFORE_Anexo 55_LIII_21 Información de las fechas de envio de los paquetes de información a detalle
17.2297 MX_BD_R24C_2431_00_22 Información de operaciones con partes relacionadas
17.2297 MX_BM_R24C_2431_00_22 Información de operaciones con partes relacionadas
17.9627 GT_BM_TCD_TCD_00_21 TCD INFORMACIÓN RELACIONADA CON TARJETAS DE CRÉDITO Y DÉBITO
18.1901 GT_BM_SFM2_SFM2_00_22 SFM2 a INFORMACIÓN DE SERVICIOS FINANCIEROS MÓVILES
18.4408 MX_IFPE_R24G_2470_00_22 INFORMACIÓN RELATIVA A CLIENTES Y OPERACIONES
18.4951 GT_BM_TCD_TC_00_21 TC INFORMACIÓN RELACIONADAS CON TARJETA DE CREDITO
18.6414 GT_BM_SFM1_SFM1_00_22 SFM1 a INFORMACIÓN DE SERVICIOS FINANCIEROS MÓVILES
18.9151 MX_CB_SAT_DECLA_PREL_00_NI Declaración Anual Informativa de Partes Relacionadas
18.9160 MX_BD_DERIVADOS_OPTO_10_22 ConvalidacioÓn de las opciones vigentes a la fecha de corte, de operaciones que fueron previamente reportadas en la SeccioÓn IX


In [ ]:
import hjson
prompt="cuáles CLAVE_REP se envían al CLAVE_REG IPAB? (VIGENTE=1)"
with open("../assets/replacements.hjson", "r") as file:
    replacements=hjson.load(file)
count=0
print(len(replacements))
values=[]
for text in replacements.values():
    values.append(text)
values=list(set(values))
for text in values:
    if text in prompt:
        count+=1
print(len(set(values)))
print(count)

4
